# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

import time

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Qiu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Qiu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Qiu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)
X = df.loc[:,'message']
Y = df.iloc[:, 4:]
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [4]:
#Define a tokenize function for later vectorization use
def tokenize(text):
    '''
    Tokenize a text message to a list of lemmatized words.
    All words will be in lowercase, singular form for noun and room form for verb
    
    Input:
    text(string): The text meassage to be tokenized
    
    Output:
    clean_words(string): List of tokenized and lemmatized words
    '''   
    text = word_tokenize(re.sub(r'[^a-zA-Z0-9]',' ',text).lower().strip())
    lemmatizer = WordNetLemmatizer()
    
    clean_words = []
    for word in text:
        clean_word = lemmatizer.lemmatize(word, pos='v')
        clean_words.append(clean_word)

    return clean_words

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [38]:
pipeline = Pipeline([
    ('tfidfvect', TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
    ('moc', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [39]:
begin = time.time()
model_1 = pipeline.fit(X_train, y_train)

f'The process of fitting cost {time.time() - begin} seconds'

'The process of fitting cost 105.74448704719543 seconds'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [59]:
y_pred = pd.DataFrame(model_1.predict(X_test), columns = category_names)

#The average of accuaracy and f1-score for all labels will be provided
tot_acc = 0
tot_f1 = 0
for i in category_names:
    #Each class label will be presented with a classification report
    print(i, 'accuracy: {:.2f}'.format(accuracy_score(y_test.loc[:, i], y_pred.loc[:, i])),'\n',
          classification_report(y_test.loc[:, i], y_pred.loc[:, i]),'\n')
    tot_acc += accuracy_score(y_test.loc[:, i], y_pred.loc[:, i])
    tot_f1 += precision_recall_fscore_support(y_test.loc[:, i], y_pred.loc[:, i], average = 'weighted')[2]
f'The average accuracy score for all labels is {round(tot_acc/len(category_names),4)},\
average f1-score for all labels is {round(tot_f1/len(category_names),4)}'

related accuracy: 0.80 
              precision    recall  f1-score   support

          0       0.63      0.49      0.55      1266
          1       0.85      0.91      0.88      3938
          2       0.54      0.17      0.26        40

avg / total       0.79      0.80      0.79      5244
 

request accuracy: 0.88 
              precision    recall  f1-score   support

          0       0.89      0.97      0.93      4349
          1       0.77      0.42      0.55       895

avg / total       0.87      0.88      0.87      5244
 

offer accuracy: 1.00 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244
 

aid_related accuracy: 0.75 
              precision    recall  f1-score   support

          0       0.76      0.84      0.80      3113
          1       0.72      0.61      0.66      2131

avg / total       0.74      0.75 

E:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



              precision    recall  f1-score   support

          0       0.96      0.99      0.98      4833
          1       0.89      0.50      0.64       411

avg / total       0.95      0.96      0.95      5244
 

storm accuracy: 0.93 
              precision    recall  f1-score   support

          0       0.94      0.98      0.96      4758
          1       0.73      0.42      0.53       486

avg / total       0.92      0.93      0.92      5244
 

fire accuracy: 0.99 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00      5191
          1       1.00      0.02      0.04        53

avg / total       0.99      0.99      0.99      5244
 

earthquake accuracy: 0.96 
              precision    recall  f1-score   support

          0       0.97      0.99      0.98      4766
          1       0.88      0.70      0.78       478

avg / total       0.96      0.96      0.96      5244
 

cold accuracy: 0.98 
              precision    recall  f

'The average accuracy score for all labels is 0.9444,average f1-score for all labels is 0.9327'

### 6. Improve your model
Use grid search to find better parameters. 

In [60]:
parameters_1 = {
        'tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'tfidfvect__max_df': [0.75, 1.0],
        'tfidfvect__max_features': [2000, 5000],
        'moc__estimator__n_estimators': [10, 15],
        'moc__estimator__max_depth': [None, 15],
        'moc__estimator__min_samples_split': [2, 5]
        }

cv = GridSearchCV(pipeline, param_grid=parameters_1)

In [61]:
begin = time.time()
model_2 = cv.fit(X_train, y_train)
f'The process of fitting cost {time.time() - begin} seconds'

'The process of fitting cost 7653.058305263519 seconds'

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [63]:
y_pred_2 = pd.DataFrame(model_2.predict(X_test), columns = category_names)

tot_acc = 0
tot_f1 = 0
for i in category_names:
    #Each class label will be presented with a classification report
    print(i, 'accuracy: {:.2f}'.format(accuracy_score(y_test.loc[:, i], y_pred_2.loc[:, i])),'\n',
          classification_report(y_test.loc[:, i], y_pred_2.loc[:, i]),'\n')
    tot_acc += accuracy_score(y_test.loc[:, i], y_pred_2.loc[:, i])
    tot_f1 += precision_recall_fscore_support(y_test.loc[:, i], y_pred_2.loc[:, i], average = 'weighted')[2]
f'The average accuracy score for all labels is {round(tot_acc/len(category_names),4)},\
average f1-score for all labels is {round(tot_f1/len(category_names),4)}'

related accuracy: 0.80 
              precision    recall  f1-score   support

          0       0.64      0.48      0.55      1266
          1       0.84      0.91      0.88      3938
          2       0.67      0.15      0.24        40

avg / total       0.79      0.80      0.79      5244
 

request accuracy: 0.89 
              precision    recall  f1-score   support

          0       0.90      0.97      0.93      4349
          1       0.76      0.49      0.60       895

avg / total       0.88      0.89      0.88      5244
 

offer accuracy: 1.00 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244
 

aid_related accuracy: 0.77 
              precision    recall  f1-score   support

          0       0.80      0.82      0.81      3113
          1       0.73      0.70      0.71      2131

avg / total       0.77      0.77 

E:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



floods accuracy: 0.96 
              precision    recall  f1-score   support

          0       0.96      0.99      0.98      4833
          1       0.84      0.56      0.67       411

avg / total       0.95      0.96      0.95      5244
 

storm accuracy: 0.94 
              precision    recall  f1-score   support

          0       0.96      0.97      0.97      4758
          1       0.71      0.62      0.66       486

avg / total       0.94      0.94      0.94      5244
 

fire accuracy: 0.99 
              precision    recall  f1-score   support

          0       0.99      1.00      0.99      5191
          1       0.00      0.00      0.00        53

avg / total       0.98      0.99      0.98      5244
 

earthquake accuracy: 0.97 
              precision    recall  f1-score   support

          0       0.98      0.99      0.98      4766
          1       0.87      0.81      0.84       478

avg / total       0.97      0.97      0.97      5244
 

cold accuracy: 0.98 
             

'The average accuracy score for all labels is 0.9484,average f1-score for all labels is 0.9397'

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [64]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    '''
    Add the length of the text message as a feature to dataset
    
    The assumption is people who is in urgent disaster condition will prefer to use less words to express
    '''
    
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(X).applymap(len)

In [65]:
# Create a new pipeline including feature union of tfidf vectorization,
pipeline_2 = Pipeline([
    ('features', FeatureUnion([       
        ('tfidfvect', TfidfVectorizer(tokenizer=tokenize)),
        ('text_length', TextLengthExtractor()) # Add a new feature
        ])),
    ('moc', MultiOutputClassifier(AdaBoostClassifier(random_state=42))) # Change the estimator to AdaBoost
    ])

In [68]:
parameters_2 = {
        'features__tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'features__tfidfvect__max_df': [0.75, 1.0],
        'features__tfidfvect__max_features': [None, 2000],
        'features__tfidfvect__use_idf': [True, False],
        'features__transformer_weights': (
            {'tfidfvect': 1, 'text_length': 0.5},
            {'tfidfvect': 1, 'text_length': 1}
        ),
        'moc__estimator__n_estimators': [30, 50],
        'moc__estimator__learning_rate': [0.5, 1]
        }

cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2)

In [69]:
begin = time.time()
model_3 = cv_2.fit(X_train, y_train)
f'The process of fitting cost {time.time() - begin} seconds'

'The process of fitting cost 41141.84708070755 seconds'

In [70]:
y_pred = pd.DataFrame(model_3.predict(X_test), columns = category_names)

tot_acc = 0
tot_f1 = 0
for i in category_names:
    #Each class label will be presented with a classification report
    print(i, 'accuracy: {:.2f}'.format(accuracy_score(y_test.loc[:, i], y_pred_1.loc[:, i])),'\n',
          classification_report(y_test.loc[:, i], y_pred_1.loc[:, i]),'\n')
    tot_acc += accuracy_score(y_test.loc[:, i], y_pred_1.loc[:, i])
    tot_f1 += precision_recall_fscore_support(y_test.loc[:, i], y_pred_1.loc[:, i], average = 'weighted')[2]
f'The average accuracy score for all labels is {round(tot_acc/len(category_names),4)},\
average f1-score for all labels is {round(tot_f1/len(category_names),4)}'

related accuracy: 0.80 
              precision    recall  f1-score   support

          0       0.63      0.49      0.55      1266
          1       0.85      0.91      0.88      3938
          2       0.54      0.17      0.26        40

avg / total       0.79      0.80      0.79      5244
 

request accuracy: 0.88 
              precision    recall  f1-score   support

          0       0.89      0.97      0.93      4349
          1       0.77      0.42      0.55       895

avg / total       0.87      0.88      0.87      5244
 

offer accuracy: 1.00 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244
 

aid_related accuracy: 0.75 
              precision    recall  f1-score   support

          0       0.76      0.84      0.80      3113
          1       0.72      0.61      0.66      2131

avg / total       0.74      0.75 

E:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'The average accuracy score for all labels is 0.9444,average f1-score for all labels is 0.9327'

### 9. Export your model as a pickle file

In [73]:
# The model2 has the highest score so it will be stored
joblib.dump(model_2, 'classifier.pkl')

['classifier.pkl']

In [92]:
# Check the best arguments for the model_2
cv.best_params_

{'moc__estimator__max_depth': None,
 'moc__estimator__min_samples_split': 5,
 'moc__estimator__n_estimators': 15,
 'tfidfvect__max_df': 0.75,
 'tfidfvect__max_features': 2000,
 'tfidfvect__ngram_range': (1, 2)}

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.